In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/Colab\ Notebooks/skema/ASKEM\ Grounding\ Annotations/
!ls

/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations
dataset
grounding_extractions_documents_5Febbuckymodel_webdocs--COSMOS-data.xlsx
grounding_extractions_documents_5FebCHIME_SIR--COSMOS-data.xlsx
grounding_extractions_documents_5FebCHIME_SVIIvR--COSMOS-data.xlsx
grounding_extractions_documents_5Febs00285-015-0906-z--COSMOS-data.xlsx
role_based_counts_buckymodel_webdocs.csv
role_based_counts_buckymodel_webdocs.gsheet
role_based_counts_CHIME_SIR.csv
role_based_counts_CHIME_SIR.gsheet
role_based_counts_CHIME_SVIIvR.csv
role_based_counts_CHIME_SVIIvR.gsheet
role_based_counts_s00285-015-0906-z.csv


In [4]:
path = "/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations"
files = [os.path.join(path, file) for file in os.listdir(path) if "COSMOS-data.xlsx" in file]
files

['/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5Febbuckymodel_webdocs--COSMOS-data.xlsx',
 '/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5Febs00285-015-0906-z--COSMOS-data.xlsx',
 '/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5FebCHIME_SIR--COSMOS-data.xlsx',
 '/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5FebCHIME_SVIIvR--COSMOS-data.xlsx']

In [36]:
data = {file:[] for file in files}
# print(data.keys())
for file in files:
  print(file, "\n",len(data[file]))
  df = pd.read_excel(file)
  df['Annotation'] = df['Annotation'].fillna("None")
  df = df.fillna(method='ffill', axis=0)
  df.insert(0, 'id', range(1, 1 + len(df)))
  i = 0
  for index, this_row in df.iterrows():
    res = []
    this_row_list = this_row.values.tolist()
    rows = df[df["Text"] == this_row['Text']][['Text','Context',"Candidate ID",	"Candidate Text","Annotation"]]

    this_list = rows[['Text','Context',"Candidate Text","Annotation"]]
    cat_values = {"Annotation":     {"X": 1, "G": 2, "None":3}}
    this_list = this_list.replace(cat_values).sort_values(by=["Annotation"], ascending=True)
    l = this_list[['Text','Context',"Candidate Text","Annotation"]].values.tolist()
    res = [{"Text":a[0], "Context": a[1], "1":a[2], "2":b[2]} for idx, a in enumerate(l) for b in l[idx + 1:] if a[3] < b[3]]
    # print(this_list[["Candidate Text","Annotation"]].values.tolist(), len(this_list.index) )
    data[file].extend(res)
  print(len(data[file]))

/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5Febbuckymodel_webdocs--COSMOS-data.xlsx 
 0
140281
/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5Febs00285-015-0906-z--COSMOS-data.xlsx 
 0
0
/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5FebCHIME_SIR--COSMOS-data.xlsx 
 0
151809
/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5FebCHIME_SVIIvR--COSMOS-data.xlsx 
 0
210


In [35]:
res

[{'Text': 'with 4 rates',
  'Candidate Text': 'with 4 rates : beta , vr , vs , gamma .',
  '1': 'Rate',
  '2': 'CEL-170/4'},
 {'Text': 'with 4 rates',
  'Candidate Text': 'with 4 rates : beta , vr , vs , gamma .',
  '1': 'Rate',
  '2': 'rate of cases with at least one comorbidity'},
 {'Text': 'with 4 rates',
  'Candidate Text': 'with 4 rates : beta , vr , vs , gamma .',
  '1': 'Rate',
  '2': 'device-specific rates and procedure-specific rates'},
 {'Text': 'with 4 rates',
  'Candidate Text': 'with 4 rates : beta , vr , vs , gamma .',
  '1': 'Rate',
  '2': 'human age 4 to 6 years at vaccination'},
 {'Text': 'with 4 rates',
  'Candidate Text': 'with 4 rates : beta , vr , vs , gamma .',
  '1': 'Rate',
  '2': 'mean with confidence interval'},
 {'Text': 'with 4 rates',
  'Candidate Text': 'with 4 rates : beta , vr , vs , gamma .',
  '1': 'Rate',
  '2': 'human age 12 to 23 months at vaccination'},
 {'Text': 'with 4 rates',
  'Candidate Text': 'with 4 rates : beta , vr , vs , gamma .',
  '1': 

In [23]:
df['Text'].unique()

array(['with 4 rates',
       'that occur in more than one term in the same equation', 'gamma2',
       'beta1', 'gamma1', 'vr', 'beta1 =', 'vs1 ='], dtype=object)

In [37]:
if "dataset" not in os.listdir(path):
  os.mkdir(os.path.join(path, "dataset"))
output_path = os.path.join(path, "dataset")
total_data_size = 0
for k,this_data in data.items():
  print(k)
  print(this_data[:1])
  print( len(this_data))
  total_data_size += len(this_data)
  file_name = os.path.basename(k)[:-18].replace("grounding_extractions_documents_", "ged_")
  pd.DataFrame.from_records(this_data).to_csv(os.path.join(output_path, file_name+".csv"))
  # pd.DataFrame.from_records(this_data).to_csv(os.path.join(output_path, file_name+".xlsx")
print(total_data_size)

/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5Febbuckymodel_webdocs--COSMOS-data.xlsx
[{'Text': 'for running simulations', 'Context': 'for running simulations', '1': 'simulation', '2': 'simulation run length'}]
140281
/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5Febs00285-015-0906-z--COSMOS-data.xlsx
[]
0
/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5FebCHIME_SIR--COSMOS-data.xlsx
[{'Text': 'for Epidemics', 'Context': 'The CHIME ( COVID-19 Hospital Impact Model for Epidemics ) App is designed to assist hospitals and public health officials understand hospital capacity needs as they relate to the COVID-19 pandemic .', '1': 'epidemic', '2': 'count of cases in an epidemic'}]
151809
/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5FebCHIME_SVIIvR--COSMOS-d

In [38]:
data.keys()

dict_keys(['/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5Febbuckymodel_webdocs--COSMOS-data.xlsx', '/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5Febs00285-015-0906-z--COSMOS-data.xlsx', '/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5FebCHIME_SIR--COSMOS-data.xlsx', '/content/drive/MyDrive/Colab Notebooks/skema/ASKEM Grounding Annotations/grounding_extractions_documents_5FebCHIME_SVIIvR--COSMOS-data.xlsx'])

In [ ]:
l = [("a",1), ("b",1), 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3]
res = [(a, b) for idx, a in enumerate(l) for b in l[idx + 1:] if a < b]
res

In [ ]:
l = [['distribution', 1], ['human daily movement data set', 2], ['Special Populations Networks', 2],
 ['triangular distribution', 3], ['continuous uniform distribution', 3], ['uniform distribution', 3],
  ['discrete probability distribution', 3], ['discrete parametric probability distribution', 3],
   ['discrete nonparametric probability distribution', 3], ['continuous parametric probability distribution', 3],
    ['close individual places independently strategy', 3], ['synthetic ecosystem data set', 3],
     ['continuous nonparametric probability distribution', 3], ['dataset creating', 3], ['United Network for Organ Sharing', 3]]
res1 = [(a, b) for idx, a in enumerate(l) for b in l[idx + 1:] if a[1] < b[1]]
res2 = [(a[0], b[0]) for idx, a in enumerate(l) for b in l[idx + 1:] if a[1] < b[1]]
res1, res2